In [7]:
from USTs import USTs
from datetime import datetime, timedelta
import QuantLib as ql
import numpy as np

In [2]:
# Bill pricing
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1

0.8

In [3]:
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

0.813

In [4]:
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [37]:
issue_date = datetime(2024, 8, 15).date()
maturity_date = datetime(2034, 8, 15).date()
coupon = 4.375
price = 100.34375
FV = 100

def create_date_set(issue_date, maturity_date):
    DAY = 15
    tenor = maturity_date.year - issue_date.year
    coupons = tenor * 2
    today = datetime.now().date()
    payment_set = []
    if issue_date.month + 6 > 12:
        coupon_months = [issue_date.month - 6, issue_date.month]
        while len(payment_set) < coupons:
            for year in range(issue_date.year + 1, maturity_date.year + 1):
                for month in coupon_months:
                    payment_set.append(datetime(year, month, DAY).date())
    else:
        coupon_months = [issue_date.month + 6, issue_date.month]
        payment_set.append(datetime(issue_date.year, coupon_months[0], DAY).date())
        for year in range(issue_date.year + 1, maturity_date.year + 1):
            for month in reversed(coupon_months):
                if len(payment_set) < 20:
                    payment_set.append(datetime(year, month, DAY).date())
    if len(payment_set) == coupons:
        print(f"{coupons} coupons dates in set, matching the expected number.")
        return payment_set

In [38]:
abc = create_date_set(issue_date=issue_date, maturity_date=maturity_date)

20 coupons dates in set, matching the expected number.


In [39]:
abc

[datetime.date(2025, 2, 15),
 datetime.date(2025, 8, 15),
 datetime.date(2026, 2, 15),
 datetime.date(2026, 8, 15),
 datetime.date(2027, 2, 15),
 datetime.date(2027, 8, 15),
 datetime.date(2028, 2, 15),
 datetime.date(2028, 8, 15),
 datetime.date(2029, 2, 15),
 datetime.date(2029, 8, 15),
 datetime.date(2030, 2, 15),
 datetime.date(2030, 8, 15),
 datetime.date(2031, 2, 15),
 datetime.date(2031, 8, 15),
 datetime.date(2032, 2, 15),
 datetime.date(2032, 8, 15),
 datetime.date(2033, 2, 15),
 datetime.date(2033, 8, 15),
 datetime.date(2034, 2, 15),
 datetime.date(2034, 8, 15)]